In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import tensorflow as tf
import malaya_speech
import malaya_speech.train
from malaya_speech.train.model import unet
from malaya_speech.utils import tf_featurization
from tensorflow.keras.layers import Multiply
import IPython.display as ipd
import numpy as np






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [5]:
def get_stft(X):
    batch_size = tf.shape(X)[0]
    stft_X = tf.TensorArray(dtype = tf.complex64, size = batch_size, dynamic_size = True, infer_shape = False)
    D_X = tf.TensorArray(dtype = tf.float32, size = batch_size, dynamic_size = True, infer_shape = False)

    init_state = (0, stft_X, D_X)
    def condition(i, features, features_len):
        return i < batch_size

    def body(i, features, features_len):
        stft_x, D_x = tf_featurization.get_stft(X[i])
        return i + 1, stft_X.write(i, stft_x), D_X.write(i, D_x)

    _, stft_X, D_X = tf.while_loop(condition, body, init_state)
    stft_X = stft_X.stack()
    stft_X.set_shape((None, None, 2049, 1))
    D_X = D_X.stack()
    D_X.set_shape((None, None, 512, 1024, 1))
    return stft_X, D_X

class Model:
    def __init__(self, size = 4):
        self.X = tf.placeholder(tf.float32, (None, None))
        self.Y = tf.placeholder(tf.float32, (None, size, None))
        
        stft_X, D_X = get_stft(self.X)
        
        self.stft = []
        for i in range(size):
            self.stft.append(get_stft(self.Y[:, i]))
        
        self.outputs = []
        for i in range(size):
            with tf.variable_scope(f'model_{i}'):
                output = unet.Model3D(D_X, dropout = 0.0, training = False).logits
                self.outputs.append(output)
        
        self.loss = []
        for i in range(size):
            self.loss.append(tf.reduce_mean(tf.abs(self.outputs[i] - self.stft[i][1])))        
        
        self.cost = tf.reduce_sum(self.loss)
        
        separation_exponent = 2
        EPSILON = 1e-10
        
        output_sum = tf.reduce_sum([o[0] ** separation_exponent for o in self.outputs], axis=0) + EPSILON
        
        self.istft = []
        for no, D in enumerate(self.outputs):
            D = D[0]

            instrument_mask = (D ** separation_exponent + (EPSILON / size)) / output_sum
            instrument_mask = tf_featurization.extend_mask(instrument_mask)
            old_shape = tf.shape(instrument_mask)
            new_shape = tf.concat(
                [[old_shape[0] * old_shape[1]], old_shape[2:]],
                axis=0)
            instrument_mask = tf.reshape(instrument_mask, new_shape)

            instrument_mask = instrument_mask[:tf.shape(stft_X[0])[0]]
            masked_stft = tf.cast(instrument_mask, dtype=tf.complex64) * stft_X[0]
            self.istft.append(tf_featurization.istft(masked_stft, self.X)[:,0])

In [6]:
tf.compat.v1.reset_default_graph()
model = Model()
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [7]:
model.outputs, model.istft

([<tf.Tensor 'model_0/output/mul:0' shape=(?, ?, 512, 1024, 1) dtype=float32>,
  <tf.Tensor 'model_1/output/mul:0' shape=(?, ?, 512, 1024, 1) dtype=float32>,
  <tf.Tensor 'model_2/output/mul:0' shape=(?, ?, 512, 1024, 1) dtype=float32>,
  <tf.Tensor 'model_3/output/mul:0' shape=(?, ?, 512, 1024, 1) dtype=float32>],
 [<tf.Tensor 'strided_slice_26:0' shape=(?,) dtype=float32>,
  <tf.Tensor 'strided_slice_40:0' shape=(?,) dtype=float32>,
  <tf.Tensor 'strided_slice_54:0' shape=(?,) dtype=float32>,
  <tf.Tensor 'strided_slice_68:0' shape=(?,) dtype=float32>])

In [ ]:
# y, sr = malaya_speech.load('../speech/example-speaker/husein-zolkepli.wav', sr = 44100)
# len(y) / sr

In [ ]:
# ipd.Audio(y, rate = sr)

In [ ]:
# y_ = malaya_speech.augmentation.waveform.sox_augment_high(y, min_bass_gain = 70,
#                                                           reverberance = 50, 
#                                                           negate = 0)
# ipd.Audio(y_, rate = sr)

In [ ]:
# noise = y - y_

In [ ]:
# stft = sess.run(model.stft, feed_dict = {model.Y: [y, noise]})

In [ ]:
# stft[0][1].shape

In [ ]:
# outputs = sess.run(model.outputs, feed_dict = {model.X: y_})
# [o.shape for o in outputs]

In [ ]:
# sess.run(model.loss, feed_dict = {model.X: y_, model.Y: [y, noise]})

In [ ]:
# istft = sess.run(model.istft, feed_dict = {model.X: y_})
# [s.shape for s in istft]

In [ ]:
# ipd.Audio(istft[0], rate = sr)

In [ ]:
# ipd.Audio(istft[1], rate = sr)

In [ ]:
# ipd.Audio(y_, rate = sr)

In [ ]:
saver = tf.train.Saver()

In [ ]:
saver.save(sess, 'test/model.ckpt')

In [ ]:
!ls -lh test

In [ ]:
tf.trainable_variables()

In [ ]:
!rm -rf test